In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob
from sklearn.preprocessing import FunctionTransformer
from sklearn.utils import shuffle


In [3]:
# Path to folder with all CSVs
folder_path = '/Users/user/Downloads/Table/downloaded_csvs_merge'

# Find all CSV files
csv_files = glob(os.path.join(folder_path, '*.csv'))

# Load and concatenate all CSVs
df_list = []
for file in csv_files:
    try:
        df_chunk = pd.read_csv(file)
        df_list.append(df_chunk)
    except Exception as e:
        print(f"Failed to load {file}: {e}")

df = pd.concat(df_list, ignore_index=True)
print("Combined Data Shape:", df.shape)


Combined Data Shape: (45019243, 40)


In [5]:
threshold = 0.9
to_drop = []

for col in df.columns:
    if df[col].isnull().mean() > threshold or (df[col] == 0).mean() > threshold:
        to_drop.append(col)

df = df.drop(columns=to_drop)
print(f"Remaining features: {df.shape[1]}")


Remaining features: 29


In [7]:
df = df.dropna()
df.reset_index(drop=True, inplace=True)
print("Shape after dropping nulls:", df.shape)


Shape after dropping nulls: (45018564, 29)


In [9]:
print(df['Label'].value_counts())


Label
DDOS-ICMP_FLOOD            6893203
DDOS-UDP_FLOOD             5180997
DDOS-TCP_FLOOD             4306058
DDOS-PSHACK_FLOOD          3920337
DDOS-SYN_FLOOD             3886105
DDOS-RSTFINFLOOD           3872787
DDOS-SYNONYMOUSIP_FLOOD    3445630
DOS-UDP_FLOOD              3177280
DOS-TCP_FLOOD              2558232
DOS-SYN_FLOOD              1942159
BENIGN                     1051313
MIRAI-GREETH_FLOOD          949325
MIRAI-UDPPLAIN              852632
MIRAI-GREIP_FLOOD           719604
DDOS-ICMP_FRAGMENTATION     433115
VULNERABILITYSCAN           357579
MITM-ARPSPOOFING            294451
DDOS-UDP_FRAGMENTATION      274881
DDOS-ACK_FRAGMENTATION      272767
DNS_SPOOFING                171463
RECON-HOSTDISCOVERY         128676
RECON-OSSCAN                 93966
RECON-PORTSCAN               78729
DOS-HTTP_FLOOD               68798
DDOS-HTTP_FLOOD              27597
DDOS-SLOWLORIS               22399
DICTIONARYBRUTEFORCE         12522
BROWSERHIJACKING              5630
COMMANDINJECTI

In [11]:
# 🔹 Step 1: Map detailed attack types to broad classes
label_map = {
    'BENIGN': 'BENIGN',
    'DDOS-ICMP_FLOOD': 'DDOS',
    'DDOS-UDP_FLOOD': 'DDOS',
    'DDOS-TCP_FLOOD': 'DDOS',
    'DDOS-PSHACK_FLOOD': 'DDOS',
    'DDOS-SYN_FLOOD': 'DDOS',
    'DDOS-RSTFINFLOOD': 'DDOS',
    'DDOS-SYNONYMOUSIP_FLOOD': 'DDOS',
    'DDOS-ICMP_FRAGMENTATION': 'DDOS',
    'DDOS-UDP_FRAGMENTATION': 'DDOS',
    'DDOS-ACK_FRAGMENTATION': 'DDOS',
    'DDOS-HTTP_FLOOD': 'DDOS',
    'DDOS-SLOWLORIS': 'DDOS',
    'DOS-UDP_FLOOD': 'NON-DDOS',
    'DOS-TCP_FLOOD': 'NON-DDOS',
    'DOS-SYN_FLOOD': 'NON-DDOS',
    'DOS-HTTP_FLOOD': 'NON-DDOS',
    'MIRAI-GREETH_FLOOD': 'NON-DDOS',
    'MIRAI-UDPPLAIN': 'NON-DDOS',
    'MIRAI-GREIP_FLOOD': 'NON-DDOS',
    'VULNERABILITYSCAN': 'NON-DDOS',
    'MITM-ARPSPOOFING': 'NON-DDOS',
    'DNS_SPOOFING': 'NON-DDOS',
    'RECON-HOSTDISCOVERY': 'NON-DDOS',
    'RECON-OSSCAN': 'NON-DDOS',
    'RECON-PORTSCAN': 'NON-DDOS',
    'DICTIONARYBRUTEFORCE': 'NON-DDOS',
    'BROWSERHIJACKING': 'NON-DDOS',
    'COMMANDINJECTION': 'NON-DDOS',
    'SQLINJECTION': 'NON-DDOS',
    'XSS': 'NON-DDOS',
    'BACKDOOR_MALWARE': 'NON-DDOS',
    'RECON-PINGSWEEP': 'NON-DDOS',
    'UPLOADING_ATTACK': 'NON-DDOS'
}

df['BroadLabel'] = df['Label'].map(label_map)

# 🔹 Step 2: Check for any unmapped labels
if df['BroadLabel'].isnull().sum() > 0:
    print("⚠️ Unmapped labels:")
    print(df[df['BroadLabel'].isnull()]['Label'].value_counts())

# 🔹 Step 3: Filter out null mappings
df = df[df['BroadLabel'].notnull()]

# 🔹 Step 4: Sample 1 million from each major class
from sklearn.utils import shuffle

sampled_df = pd.DataFrame()
for broad_class in ['BENIGN', 'DDOS', 'NON-DDOS']:
    subset = df[df['BroadLabel'] == broad_class]
    sample_n = min(1_000_000, len(subset))
    sampled_subset = subset.sample(n=sample_n, random_state=42)
    sampled_df = pd.concat([sampled_df, sampled_subset], axis=0)

sampled_df = shuffle(sampled_df).reset_index(drop=True)
print("✅ Sampled dataset shape:", sampled_df.shape)
print("✅ Distribution:\n", sampled_df['BroadLabel'].value_counts())

✅ Sampled dataset shape: (3000000, 30)
✅ Distribution:
 BroadLabel
DDOS        1000000
BENIGN      1000000
NON-DDOS    1000000
Name: count, dtype: int64


In [13]:
# 🔹 Step 5: Remove duplicate rows based on feature columns
# Keep only numeric or useful features (exclude Label/BroadLabel)
feature_columns = sampled_df.select_dtypes(include=['number']).columns.tolist()
dedup_df = sampled_df.drop_duplicates(subset=feature_columns)
print("✅ Shape after deduplication:", dedup_df.shape)

✅ Shape after deduplication: (2323046, 30)


In [15]:
# 🔹 Step 6: Drop columns with all 0/null or too many nulls
threshold = 0.98  # if more than 98% are 0 or NaN
to_drop = []

for col in feature_columns:
    if sampled_df[col].isnull().mean() > threshold or (sampled_df[col] == 0).mean() > threshold:
        to_drop.append(col)

print(f"⚠️ Dropping {len(to_drop)} low-variance or null-heavy columns: {to_drop}")
clean_df = dedup_df.drop(columns=to_drop)

⚠️ Dropping 0 low-variance or null-heavy columns: []


In [17]:
# 🔹 Step 7: Log normalization (on numeric columns only)
import numpy as np

numeric_cols = clean_df.select_dtypes(include=['number']).columns
for col in numeric_cols:
    clean_df[col] = clean_df[col].apply(lambda x: np.log1p(x))  # log1p handles 0 safely

print("✅ Log normalization applied.")
print("✅ Final preprocessed dataset shape:", clean_df.shape)

✅ Log normalization applied.
✅ Final preprocessed dataset shape: (2323046, 30)


In [19]:
# 🔹 Save the final cleaned and normalized dataset to CSV
output_path = "0final_cleaned_dataset.csv"
clean_df.to_csv(output_path, index=False)

print(f"✅ Dataset saved to '{output_path}' with shape {clean_df.shape}")


✅ Dataset saved to '0final_cleaned_dataset.csv' with shape (2323046, 30)


In [21]:
# 📦 Step 1: Import dependencies
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, LSTM, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# ✅ Step 2: Prepare dataset for binary classification
binary_df = clean_df.copy()
binary_df['BinaryLabel'] = binary_df['Label'].apply(lambda x: 'BENIGN' if x == 'BENIGN' else 'ATTACK')

# 🧪 Step 3: Split features and labels
X = binary_df.drop(columns=['Label', 'BinaryLabel'])
y = binary_df['BinaryLabel']

# 🔁 Encode target
le = LabelEncoder()
y = le.fit_transform(y)  # BENIGN → 0, ATTACK → 1

# 🧼 Step 4: Normalize features (already log normalized, just reshape)
X = X.values
X = X.reshape((X.shape[0], X.shape[1], 1))  # Shape: (samples, features, 1)

# 🔪 Step 5: Split dataset
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"Train: {X_train.shape}, Validation: {X_val.shape}, Test: {X_test.shape}")

# 🧠 Step 6: Build LSTM + Conv1D model
model = Sequential([
    Input(shape=(X.shape[1], 1)),
    Conv1D(128, kernel_size=3, activation='relu'),
    LSTM(256, return_sequences=True),
    LSTM(512),
    BatchNormalization(),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary output
])

# ⚙️ Step 7: Compile model
optimizer = Adam(learning_rate=2e-4)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 📟 Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=256,
    callbacks=[tf.keras.callbacks.ModelCheckpoint("best_model1.h5", save_best_only=True, monitor="val_accuracy", mode="max", verbose=1)]
)

# 🧪 Step 9: Evaluate on test set
model.load_weights("best_model1.h5")
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"✅ Test Accuracy: {test_accuracy:.4f}")

# 🧾 Step 10: Classification report
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred, target_names=le.classes_))


2025-05-06 20:37:22.966304: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Train: (1393827, 29, 1), Validation: (464609, 29, 1), Test: (464610, 29, 1)


ValueError: Invalid dtype: object

In [35]:
# 📦 Step 1: Import dependencies
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, LSTM, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Step 2: Create Binary Labels (Benign vs Attack)
df['Binary_Label'] = df['Label'].apply(lambda x: 'BENIGN' if x == 'BENIGN' else 'ATTACK')

# Confirm the label distribution
print(df['Binary_Label'].value_counts())


Binary_Label
ATTACK    43967251
BENIGN     1051313
Name: count, dtype: int64


In [37]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['Binary_Label_encoded'] = label_encoder.fit_transform(df['Binary_Label'])

# BENIGN → 0, ATTACK → 1
print(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))


{'ATTACK': 0, 'BENIGN': 1}


In [ ]:
# Drop unnecessary columns
columns_to_drop = ['Label', 'BroadLabel', 'Binary_Label']
feature_columns = df.drop(columns=columns_to_drop + ['Binary_Label_encoded']).select_dtypes(include=['number']).columns.tolist()

# Final feature set and label
X = df[feature_columns]
y = df['Binary_Label_encoded']

print("✅ Feature shape:", X.shape)


In [29]:
# 🧠 Step 6: Build LSTM + Conv1D model
model = Sequential([
    Input(shape=(X.shape[1], 1)),
    Conv1D(128, kernel_size=3, activation='relu'),
    LSTM(256, return_sequences=True),
    LSTM(512),
    BatchNormalization(),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary output
])


In [31]:
# ⚙️ Step 7: Compile model
optimizer = Adam(learning_rate=2e-4)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [33]:
# 📟 Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=256,
    callbacks=[tf.keras.callbacks.ModelCheckpoint("best_model1.h5", save_best_only=True, monitor="val_accuracy", mode="max", verbose=1)]
)

# 🧪 Step 9: Evaluate on test set
model.load_weights("best_model1.h5")
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"✅ Test Accuracy: {test_accuracy:.4f}")

ValueError: Invalid dtype: object

In [1]:
import pandas as pd
df = pd.read_csv('/Users/user/Downloads/0final_cleaned_dataset.csv')
df.head()  # Shows the first 5 rows


,Header_Length,Protocol Type,Time_To_Live,Rate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,ack_flag_number,ack_count,...,Min,Max,AVG,Std,Tot size,IAT,Number,Variance,Label,BroadLabel
0,2.197225,2.890372,4.174387,10.633350,0.0,0.000000,0.0,0.00000,0.000000,0.000000,...,4.110874,4.110874,4.110874,0.000000,4.110874,0.000024,4.615121,0.000000,DDOS-UDP_FLOOD,DDOS
1,3.034953,1.945910,4.164492,9.500621,0.0,0.688135,0.0,0.00000,0.000000,0.000000,...,4.110874,4.110874,4.110874,0.000000,4.110874,0.000075,4.615121,0.000000,DDOS-SYNONYMOUSIP_FLOOD,DDOS
2,3.496508,1.945910,4.174387,6.768015,0.0,0.000000,0.0,0.00000,0.693147,2.397895,...,7.323171,8.391857,7.575277,6.885842,7.575277,0.001308,2.397895,13.769641,BENIGN,BENIGN
3,2.232163,2.890372,4.174387,10.137531,0.0,0.009950,0.0,0.00000,0.000000,0.000000,...,4.110874,4.317488,4.113166,0.875469,4.113166,0.000040,4.615121,1.085189,DOS-UDP_FLOOD,NON-DDOS
4,3.496508,1.945910,5.285739,4.848298,0.0,0.000000,0.0,0.09531,0.693147,2.397895,...,4.204693,5.153292,4.351567,3.541543,4.351567,0.009918,2.397895,7.025183,BENIGN,BENIGN


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, LSTM, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

2025-05-06 22:51:38.644950: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# Step 2: Create Binary Labels (Benign vs Attack)
df['Binary_Label'] = df['Label'].apply(lambda x: 'BENIGN' if x == 'BENIGN' else 'ATTACK')

# Confirm the label distribution
print(df['Binary_Label'].value_counts())


Binary_Label
ATTACK    1326940
BENIGN     996106
Name: count, dtype: int64


In [7]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['Binary_Label_encoded'] = label_encoder.fit_transform(df['Binary_Label'])

# BENIGN → 0, ATTACK → 1
print(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))


{'ATTACK': 0, 'BENIGN': 1}


In [9]:
# Drop unnecessary columns
columns_to_drop = ['Label', 'BroadLabel', 'Binary_Label']
feature_columns = df.drop(columns=columns_to_drop + ['Binary_Label_encoded']).select_dtypes(include=['number']).columns.tolist()

# Final feature set and label
X = df[feature_columns]
y = df['Binary_Label_encoded']

print("✅ Feature shape:", X.shape)


✅ Feature shape: (2323046, 28)


In [11]:
from sklearn.model_selection import train_test_split

# Split: 60% train, 20% val, 20% test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"✅ Train: {X_train.shape}, Validation: {X_val.shape}, Test: {X_test.shape}")


✅ Train: (1393827, 28), Validation: (464609, 28), Test: (464610, 28)


In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("✅ Data scaled.")


ValueError: Input X contains infinity or a value too large for dtype('float64').

In [15]:
# Check for NaNs and infinite values
print("NaNs in X_train:", np.isnan(X_train).sum().sum())
print("Infs in X_train:", np.isinf(X_train).sum().sum())


NaNs in X_train: 0
Infs in X_train: 5


In [17]:
# Replace inf with NaN, then fill NaN with mean
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_val = X_val.replace([np.inf, -np.inf], np.nan)
X_test = X_test.replace([np.inf, -np.inf], np.nan)

# Fill missing values with column mean
X_train = X_train.fillna(X_train.mean())
X_val = X_val.fillna(X_train.mean())  # Use training mean for validation/test
X_test = X_test.fillna(X_train.mean())


In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("✅ Data scaled.")


✅ Data scaled.


In [21]:
import numpy as np

X_train_seq = np.expand_dims(X_train_scaled, axis=-1)
X_val_seq = np.expand_dims(X_val_scaled, axis=-1)
X_test_seq = np.expand_dims(X_test_scaled, axis=-1)

print("✅ Reshaped for LSTM input:", X_train_seq.shape)


✅ Reshaped for LSTM input: (1393827, 28, 1)


In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, LSTM, BatchNormalization, Flatten, Dense, Dropout

model = Sequential([
    Input(shape=(X_train_seq.shape[1], 1)),
    Conv1D(128, kernel_size=3, activation='relu'),
    LSTM(256, return_sequences=True),
    LSTM(512),
    BatchNormalization(),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # binary classification
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 26, 128)        │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 26, 256)        │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 512)            │     1,574,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,497,537 (9.53 MB)

 Trainable params: 2,496,513 (9.52 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [25]:
history = model.fit(
    X_train_seq, y_train,
    validation_data=(X_val_seq, y_val),
    epochs=200,
    batch_size=256,
    callbacks=[tf.keras.callbacks.ModelCheckpoint("best_model1.h5", save_best_only=True, monitor="val_accuracy", mode="max", verbose=1)]
)


Epoch 1/200
5445/5445 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9604 - loss: 0.1168
Epoch 1: val_accuracy improved from -inf to 0.95732, saving model to best_model1.h5


5445/5445 ━━━━━━━━━━━━━━━━━━━━ 9743s 2s/step - accuracy: 0.9604 - loss: 0.1168 - val_accuracy: 0.9573 - val_loss: 0.1540
Epoch 2/200
5445/5445 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9671 - loss: 0.0942
Epoch 2: val_accuracy did not improve from 0.95732
5445/5445 ━━━━━━━━━━━━━━━━━━━━ 8748s 2s/step - accuracy: 0.9671 - loss: 0.0942 - val_accuracy: 0.9565 - val_loss: 0.1250
Epoch 3/200
5445/5445 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9687 - loss: 0.0897
Epoch 3: val_accuracy did not improve from 0.95732
5445/5445 ━━━━━━━━━━━━━━━━━━━━ 8509s 2s/step - accuracy: 0.9687 - loss: 0.0897 - val_accuracy: 0.9523 - val_loss: 0.1276
Epoch 4/200
5445/5445 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9711 - loss: 0.0847
Epoch 4: val_accuracy improved from 0.95732 to 0.96830, saving model to best_model1.h5


5445/5445 ━━━━━━━━━━━━━━━━━━━━ 9499s 2s/step - accuracy: 0.9711 - loss: 0.0847 - val_accuracy: 0.9683 - val_loss: 0.0975
Epoch 5/200
5445/5445 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9722 - loss: 0.0822
Epoch 5: val_accuracy improved from 0.96830 to 0.97077, saving model to best_model1.h5


5445/5445 ━━━━━━━━━━━━━━━━━━━━ 9870s 2s/step - accuracy: 0.9722 - loss: 0.0822 - val_accuracy: 0.9708 - val_loss: 0.0884
Epoch 6/200
5445/5445 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9723 - loss: 0.0814
Epoch 6: val_accuracy improved from 0.97077 to 0.97226, saving model to best_model1.h5


5445/5445 ━━━━━━━━━━━━━━━━━━━━ 9550s 2s/step - accuracy: 0.9723 - loss: 0.0814 - val_accuracy: 0.9723 - val_loss: 0.0818
Epoch 7/200
5445/5445 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9727 - loss: 0.0805
Epoch 7: val_accuracy improved from 0.97226 to 0.97246, saving model to best_model1.h5


5445/5445 ━━━━━━━━━━━━━━━━━━━━ 8985s 2s/step - accuracy: 0.9727 - loss: 0.0805 - val_accuracy: 0.9725 - val_loss: 0.0814
Epoch 8/200
5445/5445 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9733 - loss: 0.0792
Epoch 8: val_accuracy did not improve from 0.97246
5445/5445 ━━━━━━━━━━━━━━━━━━━━ 11211s 2s/step - accuracy: 0.9733 - loss: 0.0792 - val_accuracy: 0.9722 - val_loss: 0.0829
Epoch 9/200
1651/5445 ━━━━━━━━━━━━━━━━━━━━ 1:39:47 2s/step - accuracy: 0.9728 - loss: 0.0790

KeyboardInterrupt: 

In [35]:
import matplotlib.pyplot as plt
from keras.callbacks import Callback

class RealTimePlotting(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            # Clear the figure and replot at each epoch
            plt.clf()
            # Plot training accuracy and validation accuracy
            plt.subplot(1, 2, 1)
            plt.plot(self.history.history['accuracy'], label='train accuracy')
            plt.plot(self.history.history['val_accuracy'], label='val accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.legend()
            plt.title('Accuracy')

            # Plot training loss and validation loss
            plt.subplot(1, 2, 2)
            plt.plot(self.history.history['loss'], label='train loss')
            plt.plot(self.history.history['val_loss'], label='val loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()
            plt.title('Loss')

            # Display the plot
            plt.pause(0.1)  # Pause to update the plot
            plt.show()


In [33]:
# Create the callback
real_time_plotting = RealTimePlotting()

# Continue training with real-time plotting (or restart the training)
model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_val, y_val), callbacks=[real_time_plotting])


Epoch 1/200


ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=True
  • mask=None